In [10]:
import torch


points = torch.Tensor([[0, 0, 0]])
rotation = torch.eye(3)
translation = torch.Tensor([0, 0, 100])
extrinsic_matrix = torch.cat([rotation, translation.unsqueeze(1)], dim=1)
extrinsic_matrix = torch.cat([extrinsic_matrix, torch.Tensor([[0, 0, 0, 1]])], dim=0)

points_homogeneous = torch.cat([points, torch.ones(points.shape[0], 1)], dim=1)
points_camera_space = points_homogeneous @ extrinsic_matrix.T

points_camera_space

tensor([[  0.,   0., 100.,   1.]])

In [18]:
import math
from typing import Tuple
from splat.render_engine.utils import compute_fov_from_focal
from splat.utils import getIntinsicMatrix

far_clipping_plane = 10000.0
near_clipping_plane = 0.001

focal_x = 4400
focal_y = 4400
height = 200
width = 300

intrinsic_matrix = getIntinsicMatrix(focal_x, focal_y, height, width)

def compute_2d_covariance(
    points_homogeneous: torch.Tensor,
    covariance_3d: torch.Tensor,
    extrinsic_matrix: torch.Tensor,
    tan_fovX: float,
    tan_fovY: float,
    focal_x: float,
    focal_y: float,
) -> Tuple[torch.Tensor, torch.Tensor]:
    """Make sure the extrinsic matrix has the translation in the last row"""

    points_camera_space = points_homogeneous @ extrinsic_matrix
    x = points_camera_space[:, 0] / points_camera_space[:, 2]
    y = points_camera_space[:, 1] / points_camera_space[:, 2]
    x = torch.clamp(x, -1.3 * tan_fovX, 1.3 * tan_fovX) * points_camera_space[:, 2]
    y = torch.clamp(y, -1.3 * tan_fovY, 1.3 * tan_fovY) * points_camera_space[:, 2]
    z = points_camera_space[:, 2]
    print(x, y, z)

    j = torch.zeros((points_camera_space.shape[0], 2, 3)).to("cpu")
    j[:, 0, 0] = focal_x / z
    j[:, 0, 2] = -(focal_x * x) / (z**2)
    j[:, 1, 1] = focal_y / z
    j[:, 1, 2] = -(focal_y * y) / (z**2)

    # we assume our extrinsic matrix has the translation in the last row
    # so it is already transposed so we transpose back
    # overall formula for a normal extrinsic matrix is
    # J @ W @ covariance_3d @ W.T @ J.T
    covariance2d = (
        j
        @ extrinsic_matrix[:3, :3].T
        @ covariance_3d
        @ extrinsic_matrix[:3, :3]
        @ j.transpose(1, 2)
    )
    return covariance2d, points_camera_space

covariance_3d = torch.eye(3)

fovX = compute_fov_from_focal(focal_x, width)
fovY = compute_fov_from_focal(focal_y, height)
tan_fovX = math.tan(fovX / 2)
tan_fovY = math.tan(fovY / 2)

covariance2d, points_camera_space = compute_2d_covariance(points_homogeneous, covariance_3d, extrinsic_matrix.T, tan_fovX, tan_fovY, focal_x, focal_y)

covariance2d

tensor([3.4091e-05]) tensor([-3.4091e-05]) tensor([2.2727e-05]) tensor([-2.2727e-05]) tensor([0.0010]) tensor([100.])
tensor([0.]) tensor([0.]) tensor([100.])


tensor([[[1936.,    0.],
         [   0., 1936.]]])